# Demo 4: FP-Growth and DataStax Analytics
------
<img src="images/pixarMovies.jpg" width="500" height="500">


#### Dataset: https://grouplens.org/datasets/movielens/

## What are we trying to learn from this dataset? 

# QUESTION: Can FP-Growth be used to find which movies to recommend to our users?


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import pandas
import cassandra
import pyspark
import re
import os
import random
import matplotlib.pyplot as plt
from random import randint, randrange
from IPython.display import display, Markdown
from pyspark.sql import SparkSession
from pyspark.ml.fpm import FPGrowth
from pyspark.sql import Row
from pyspark.sql.functions import collect_set
from pyspark.ml.feature import IndexToString, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
#Helper for pretty formatting for Spark DataFrames
def showDF(df, limitRows =  5, truncate = True):
    if(truncate):
        pandas.set_option('display.max_colwidth', 50)
    else:
        pandas.set_option('display.max_colwidth', -1)
    pandas.set_option('display.max_rows', limitRows)
    display(df.limit(limitRows).toPandas())
    pandas.reset_option('display.max_rows')

#### Helper function to have nicer formatting of Spark DataFrames

## Creating Tables and Loading Tables

### Connect to DSE Analytics Cluster

# DataStax Enterprise Analytics
<img src="images/dselogo.png" width="400" height="200">

In [4]:
from cassandra.cluster import Cluster

cluster = Cluster(['dse'])
session = cluster.connect()

### Create Demo Keyspace 

In [8]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS accelerate 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

### Set keyspace 

In [9]:
session.set_keyspace('accelerate')

### Create table called `movies`. Our PRIMARY will be a unique key (movieid) 

In [10]:
query = "CREATE TABLE IF NOT EXISTS movies \
                                   (movieid int, title text, genres text, \
                                   PRIMARY KEY (movieid))"
session.execute(query)

### Create table called `movieRatings`. Our PRIMARY key will be a compositite key (userid, movieid).

In [11]:
query = "CREATE TABLE IF NOT EXISTS movieratings \
                                   (userid int, movieid int, rating float, timestamp text, \
                                   PRIMARY KEY (userid, movieid))"
session.execute(query)

## Movies

* **Movieid**
* **Title**
* **Genres**

## Movie Ratings Table
### What do these of these 4 columns represent:

* **UserId**
* **MovieId**
* **Rating**
* **Timestamp**

### Load 2 Movie Dataset -- Movies and Movie Ratings Table
<img src="images/bttf3.jpg" width="500" height="300">

### Load Movie datasets from CSV file (rating_movies.csv, movies.csv)
* No clean up was requried! How nice :)

#### Insert all the Movie Data into the DSE table `movies` and `movieratings`

In [12]:
fileName = 'data/ratings.csv'
input_file = open(fileName, 'r')

for line in input_file:
    row = line.split(',')
    
    query = "INSERT INTO movieratings (userid, movieid, rating, timestamp)"
    query = query + " VALUES (%s, %s, %s, %s)"
    session.execute(query, (int(row[0]), int(row[1]), float(row[2]), row[3]))

In [13]:
fileName = 'data/movies.csv'
input_file = open(fileName, 'r')

for line in input_file:
    row = line.split(',')
        
    query = "INSERT INTO movies (movieid, title, genres)"
    query = query + " VALUES (%s, %s, %s)"
    session.execute(query, (int(row[0]), row[1], row[2]))

## Machine Learning with DSE Analytics and Apache Spark
<img src="images/sparklogo.png" width="150" height="200">

#### Create a spark session that is connected to DSE. From there load each table into a Spark Dataframe and take a count of the number of rows in each.

In [14]:
spark = SparkSession.builder.appName('demo').master("local").getOrCreate()

movieDF = spark.read.format("org.apache.spark.sql.cassandra").options(table="movieratings", keyspace="accelerate").load()

print ("Table Row Count: ")
print (movieDF.count())

Table Row Count: 
100000


In [15]:
showDF(movieDF)

,userid,movieid,rating,timestamp
0,23,1,4.0,12/24/98 0:13\n
1,23,25,4.0,12/24/98 0:07\n
2,23,32,5.0,12/24/98 0:12\n
3,23,47,5.0,12/24/98 0:06\n
4,23,50,5.0,12/24/98 0:02\n


## This dataset is not in the format we need it to be. We need it to be more in a transaction format. Each user and the list of movies they have reviewed. 

#### Remove the timestamp column since we will not be using that

In [16]:
newMovieDF = movieDF.drop('timestamp')
showDF(newMovieDF)

,userid,movieid,rating
0,23,1,4.0
1,23,25,4.0
2,23,32,5.0
3,23,47,5.0
4,23,50,5.0


#### Before we collect the set of movies for each user let's filter out any movies they rated below 3

In [17]:
newestMovies = newMovieDF.filter("rating > 3")
showDF(newestMovies)

,userid,movieid,rating
0,23,1,4.0
1,23,25,4.0
2,23,32,5.0
3,23,47,5.0
4,23,50,5.0


#### GroupBy the user id and create a collection set of all the movies they have rated and seen. 

In [13]:
group_user = newestMovies.groupBy('userid').agg(collect_set('movieid').alias('moviesRated'))
group_user.show()


+------+--------------------+
|userid|         moviesRated|
+------+--------------------+
|   463|[161, 509, 590, 2...|
|   148|[356, 4995, 539, ...|
|   471|[508, 356, 4008, ...|
|   496|[356, 1953, 1395,...|
|   623|[356, 165, 593, 5...|
|   243|[592, 356, 153, 1...|
|   392|[3254, 596, 4995,...|
|   540|[356, 70286, 5874...|
|    31|[54997, 45517, 70...|
|   516|[356, 785, 1345, ...|
|   137|[356, 3173, 1222,...|
|   251|[466, 356, 110, 3...|
|    85|[945, 916, 1293, ...|
|   580|[4306, 4973, 1223...|
|   451|[6440, 1222, 714,...|
|   458|[299, 2396, 3798,...|
|    65|[356, 40870, 7609...|
|    53|[1953, 1649, 1172...|
|   255|[2, 2006, 380, 20...|
|   588|[4995, 4343, 3450...|
+------+--------------------+
only showing top 20 rows



#### For FP-Growth the list needs to be a column named `items`

In [14]:
df = group_user.withColumnRenamed("moviesRated", "items")
showDF(df)

,userid,items
0,463,"[161, 509, 590, 277, 105, 410, 207, 25, 381, 5..."
1,148,"[356, 4995, 539, 916, 2340, 4285, 1680, 4062, ..."
2,471,"[508, 356, 4008, 1272, 8982, 3450, 2078, 2028,..."
3,496,"[356, 1953, 1395, 1476, 1222, 3421, 2028, 1266..."
4,623,"[356, 165, 593, 590, 318, 292, 50, 296, 380, 3..."


In [17]:
df.select('userid').distinct().count()

702

### FPGROWTH for  Recommendations
#### Use Apache Spark MLlib with FPGrowth to find Recommendation 
#### https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html
#### https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.fpm.FPGrowth

In [15]:
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.1, minConfidence=0.2)
model = fpGrowth.fit(df)
recommendDF=model.transform(df)
recommendDF.show()

+------+--------------------+--------------------+
|userid|               items|          prediction|
+------+--------------------+--------------------+
|   463|[161, 509, 590, 2...|[50, 356, 2858, 2...|
|   148|[356, 4995, 539, ...|[4993, 2959, 296,...|
|   471|[508, 356, 4008, ...|[296, 589, 2858, ...|
|   496|[356, 1953, 1395,...|[318, 593, 296, 2...|
|   623|[356, 165, 593, 5...|[2858, 47, 260, 5...|
|   243|[592, 356, 153, 1...|[593, 260, 527, 2...|
|   392|[3254, 596, 4995,...|[356, 527, 260, 5...|
|   540|[356, 70286, 5874...|[593, 318, 589, 4...|
|    31|[54997, 45517, 70...|[356, 527, 296, 5...|
|   516|[356, 785, 1345, ...|[318, 593, 2858, ...|
|   137|[356, 3173, 1222,...|[50, 47, 5952, 49...|
|   251|[466, 356, 110, 3...|[318, 593, 457, 2...|
|    85|[945, 916, 1293, ...|[2959, 296, 2571,...|
|   580|[4306, 4973, 1223...|[318, 356, 593, 5...|
|   451|[6440, 1222, 714,...|[1198, 260, 1196,...|
|   458|[299, 2396, 3798,...|[50, 296, 858, 60...|
|    65|[356, 40870, 7609...|[1

#### If you have watched these movies `antecedent` then you will like this movie `consquent`

In [16]:
# Display frequent itemsets.
#model.freqItemsets.show()

# Display generated association rules.
dfAssociation = model.associationRules

dfAssociation.show()

+------------+----------+------------------+------------------+
|  antecedent|consequent|        confidence|              lift|
+------------+----------+------------------+------------------+
|  [608, 593]|     [296]|0.8301886792452831| 2.111566858080394|
|  [608, 593]|     [318]|0.7075471698113207| 1.793133982698726|
|[7153, 2959]|    [4993]| 0.935064935064935| 4.405473720909962|
|       [541]|    [1198]|0.5658914728682171|2.2069767441860466|
|       [541]|     [260]|0.6124031007751938|  2.01834261382247|
|       [541]|     [296]|0.6744186046511628|1.7153690596562183|
|       [541]|     [593]|0.5581395348837209|1.6124031007751938|
|       [541]|    [1196]|0.5968992248062015| 2.289744567289363|
|       [541]|    [2571]|0.6589147286821705|2.1922186707814393|
|  [527, 593]|     [356]|0.6428571428571429|1.6714285714285715|
|  [527, 593]|     [296]|0.7053571428571429|1.7940605590062113|
|  [527, 593]|     [318]|0.7142857142857143|1.8102114492006187|
| [2028, 296]|    [2858]| 0.782608695652

In [17]:
print("If you like these movies: ")
print(list(dfAssociation.select('antecedent').first()))
print("Then you will like this movie:")
print(list(dfAssociation.select('consequent').first()))

movieYoulike = list(dfAssociation.select('antecedent').first())
movieToRecommend=list(dfAssociation.select('consequent').first())

If you like these movies: 
[[608, 593]]
Then you will like this movie:
[[296]]


#### Query DSE to get movie titles

In [18]:
query = "select title from movies WHERE movieid="
query = query + str(movieYoulike[0][0])

rows = session.execute(query)
print(rows)

for user_row in rows:
    print (user_row.title)

query = "select title from movies WHERE movieid="
query = query + str(movieYoulike[0][1])

rows = session.execute(query)
print(rows)

for user_row in rows:
    print (user_row.title)

Fargo (1996)
"Silence of the Lambs


#### Then you will like this movie ... 

In [19]:
query = "select title from movies WHERE movieid="
query = query + str(movieToRecommend[0][0])

rows = session.execute(query)
print(rows)

for user_row in rows:
    print (user_row.title)

Pulp Fiction (1994)


In [ ]:
session.execute("""drop table movies""")
session.execute("""drop table movieratings""")